Ce code permet d'automatiser le rapport sur le prix au m². 

Les objets à mettre dans le rapport sont marqués par # TODO en début de cellule. (Il faut exécuter la cellule puis insérer au Latex ce qui est indiqué dans TODO)

Le code est organisé de la même manière que le rapport, les titres sont écrits dans des cellules Markdown

Executer sur l'environnement base

### A faire par l'utilisateur :
- Copier/coller les données de MeilleursAgents: https://www.meilleursagents.com/prix-immobilier/. Mettre le fichier à cet emplacement : N:\uflujek\table_rapport\Prix_m2\prix_MA_dept.xlsx (pour les départements) et N:\uflujek\table_rapport\Prix_m2\prix_MA_ville.xlsx (pour les villes)

Copier/coller directement les données dans excel et supprimer les liens hypertextes (clique droit sur la première colonne + supprimer les liens hypertextes)

In [27]:
import os
import pandas as pd
import time
import numpy as np
import textwrap
import matplotlib.pyplot as plt
import seaborn as sns
import re
import folium
import xlsxwriter
import json
from folium.features import GeoJsonTooltip
import io
import scipy.stats as stats
from datetime import datetime
import string
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.styles.differential import DifferentialStyle
from openpyxl.formatting.rule import Rule
from openpyxl.utils import get_column_letter
from openpyxl.utils.dataframe import dataframe_to_rows
import shutil
from openpyxl.drawing.image import Image
from openpyxl.styles import Font, Color, PatternFill, Border, Side
from openpyxl.utils.cell import coordinate_from_string, column_index_from_string
import warnings
import pathlib
import pyodbc
import pandasql as ps
warnings.filterwarnings('ignore')
sns.set_theme()

In [28]:
SERVEUR = 'ISIS'
DATABASE = 'BIENS'
SCHEMA = 'dbo'




PROD_PWD = 'Est1mat1on@b1ens_Dev%'
PROD_ID = 'Ebiens_Batch'



conn = pyodbc.connect('DRIVER={SQL Server};'
                                f'SERVER={SERVEUR};'
                                f'DATABASE={DATABASE};'
                                f'UID={PROD_ID};'
                                f'PWD={PROD_PWD};')
df_prix_m2 = pd.read_sql_query(
    f"""SELECT DISTINCT *
    FROM BIENS.dbo.PRIX_M2
    WHERE MAILLE_GEO = 'DEPT' AND DIC_NAME LIKE '%cty_%_%m'"""
    ,conn)
df_prix_m2

,ID,DIC_NAME,KEY_DICT,MAILLE_DICT,CD_TYP_BIEN,MAILLE_GEO,VAL_MAILLE_GEO,DT_MAJ_PM2,NB_BIENS,MEAN,STD,MEDIAN,D1,Q1,Q3,D9
0,1,__pm2_cty_3m,APT_01,cty,APT,DEPT,01,2025-09-01,19.0,2108.27,1203.65,1563.95,1062.46,1263.93,3108.17,3678.47
1,2,__pm2_cty_3m,APT_02,cty,APT,DEPT,02,2025-09-01,7.0,1185.23,491.75,931.51,771.04,791.81,1516.83,1728.30
2,3,__pm2_cty_3m,APT_03,cty,APT,DEPT,03,2025-09-01,7.0,744.97,383.20,638.66,428.76,545.96,874.89,1182.80
3,4,__pm2_cty_3m,APT_04,cty,APT,DEPT,04,2025-09-01,14.0,1321.21,484.46,1274.40,695.59,1046.78,1736.01,1943.20
4,5,__pm2_cty_3m,APT_05,cty,APT,DEPT,05,2025-09-01,7.0,3621.51,672.24,3486.49,3010.38,3329.17,4083.33,4414.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,39776,__pm2_cty_comp_12m,972,cty,ALL,DEPT,972,2025-09-01,55.0,2410.72,1295.40,2202.38,889.66,1377.22,3295.41,4262.26
863,39777,__pm2_cty_comp_12m,973,cty,ALL,DEPT,973,2025-09-01,13.0,2357.05,1532.45,1993.01,616.54,1145.83,3312.50,4525.94
864,39778,__pm2_cty_comp_12m,974,cty,ALL,DEPT,974,2025-09-01,13.0,1631.03,1286.64,1478.26,512.04,985.92,1848.48,2147.32
865,39779,__pm2_cty_comp_12m,98,cty,ALL,DEPT,98,2025-09-01,56.0,4294.89,2491.15,3667.65,1927.64,2853.87,5361.30,6784.69


In [29]:
df = pd.read_excel(r"N:\uflumth\indicateur_marche\V2\données indice de prix\table_rapport (2025)\departements francais.xlsx")
df['Numéro'] = df['Numéro'].astype(str)
df = df.drop(df.index[102:107])
df.rename(columns={"2025 (p)":"Population"}, inplace = True)
df['Population'] = pd.to_numeric(df['Population'], errors='coerce')
df['Numéro'] = df['Numéro'].apply(lambda x: x.zfill(2))
df_prix_m2 = pd.merge(df, df_prix_m2, left_on='Numéro', right_on='VAL_MAILLE_GEO', how='inner')
df_prix_m2 = df_prix_m2.drop(columns= ["Population_y","Numéro_y", "Département_y","Numéro_x", "Département_x", "Population_x", "Numéro_x", "Département_x",
                                       "ID"], errors = "ignore")


## Prix au m² des départements français

### Carte

In [30]:
df_prix_m2

,Numéro,Département,Population,DIC_NAME,KEY_DICT,MAILLE_DICT,CD_TYP_BIEN,MAILLE_GEO,VAL_MAILLE_GEO,DT_MAJ_PM2,NB_BIENS,MEAN,STD,MEDIAN,D1,Q1,Q3,D9
0,01,Ain,688626.0,__pm2_cty_3m,APT_01,cty,APT,DEPT,01,2025-09-01,19.0,2108.27,1203.65,1563.95,1062.46,1263.93,3108.17,3678.47
1,01,Ain,688626.0,__pm2_cty_3m,MAI_01,cty,MAI,DEPT,01,2025-09-01,28.0,2753.39,907.41,2853.71,1408.82,2164.81,3595.51,3708.63
2,01,Ain,688626.0,__pm2_cty_6m,APT_01,cty,APT,DEPT,01,2025-09-01,8.0,1965.25,692.05,1734.97,1271.73,1555.56,2323.25,2890.60
3,01,Ain,688626.0,__pm2_cty_6m,MAI_01,cty,MAI,DEPT,01,2025-09-01,71.0,2609.68,823.00,2669.90,1512.61,2142.26,3171.43,3601.77
4,01,Ain,688626.0,__pm2_cty_12m,APT_01,cty,APT,DEPT,01,2025-09-01,50.0,2319.64,1393.15,1759.99,1118.83,1459.12,2714.89,4052.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
841,974,La Réunion,896175.0,__pm2_cty_6m,MAI_974,cty,MAI,DEPT,974,2025-09-01,11.0,1792.04,1337.76,1500.00,540.00,1325.68,1892.55,2200.00
842,974,La Réunion,896175.0,__pm2_cty_12m,MAI_974,cty,MAI,DEPT,974,2025-09-01,13.0,1631.03,1286.64,1478.26,512.04,985.92,1848.48,2147.32
843,974,La Réunion,896175.0,__pm2_cty_comp_3m,974,cty,ALL,DEPT,974,2025-09-01,11.0,1792.04,1337.76,1500.00,540.00,1325.68,1892.55,2200.00
844,974,La Réunion,896175.0,__pm2_cty_comp_6m,974,cty,ALL,DEPT,974,2025-09-01,11.0,1792.04,1337.76,1500.00,540.00,1325.68,1892.55,2200.00


In [31]:
# On sépare les données des 3 derniers mois et des 6 derniers mois sous 2 dataframe. 

df_final = pd.DataFrame()
list_tables = [] 



df = df_prix_m2


df_3m = df[(df['DIC_NAME'] == '__pm2_cty_3m') & (df['CD_TYP_BIEN'] != 'ALL') | 
            (df['DIC_NAME'] == '__pm2_cty_comp_3m') & (df['CD_TYP_BIEN'] == 'ALL')]
df_6m = df[((df['DIC_NAME'] == '__pm2_cty_6m') & (df['CD_TYP_BIEN'] != 'ALL')) | 
            ((df['DIC_NAME'] == '__pm2_cty_comp_6m') & (df['CD_TYP_BIEN'] == 'ALL'))]



df_3m = df_3m.rename(columns={'MEAN': 'mean_3m', 'NB_BIENS': 'nb_bien_3m'})
df_6m = df_6m.rename(columns={'MEAN': 'mean_6m', 'NB_BIENS': 'nb_bien_6m'})
df_merged = pd.merge(df_3m, df_6m, on=['Département', 'CD_TYP_BIEN'], how='inner')
df_merged

,Numéro_x,Département,Population_x,DIC_NAME_x,KEY_DICT_x,MAILLE_DICT_x,CD_TYP_BIEN,MAILLE_GEO_x,VAL_MAILLE_GEO_x,DT_MAJ_PM2_x,...,VAL_MAILLE_GEO_y,DT_MAJ_PM2_y,nb_bien_6m,mean_6m,STD_y,MEDIAN_y,D1_y,Q1_y,Q3_y,D9_y
0,01,Ain,688626.0,__pm2_cty_3m,APT_01,cty,APT,DEPT,01,2025-09-01,...,01,2025-09-01,8.0,1965.25,692.05,1734.97,1271.73,1555.56,2323.25,2890.60
1,01,Ain,688626.0,__pm2_cty_3m,MAI_01,cty,MAI,DEPT,01,2025-09-01,...,01,2025-09-01,71.0,2609.68,823.00,2669.90,1512.61,2142.26,3171.43,3601.77
2,01,Ain,688626.0,__pm2_cty_comp_3m,01,cty,ALL,DEPT,01,2025-09-01,...,01,2025-09-01,77.0,2547.91,832.00,2578.95,1393.65,2060.00,3142.86,3599.37
3,02,Aisne,518817.0,__pm2_cty_3m,APT_02,cty,APT,DEPT,02,2025-09-01,...,02,2025-09-01,7.0,1373.44,299.31,1475.00,1059.13,1353.33,1516.83,1588.68
4,02,Aisne,518817.0,__pm2_cty_3m,MAI_02,cty,MAI,DEPT,02,2025-09-01,...,02,2025-09-01,88.0,1491.59,601.18,1452.22,829.13,1095.26,1791.49,2205.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,972,Martinique,355459.0,__pm2_cty_comp_3m,972,cty,ALL,DEPT,972,2025-09-01,...,972,2025-09-01,7.0,2431.68,1277.90,2202.38,1294.96,1455.78,3082.61,3885.80
278,973,Guyane,292354.0,__pm2_cty_3m,MAI_973,cty,MAI,DEPT,973,2025-09-01,...,973,2025-09-01,9.0,2107.49,1748.56,1883.12,611.01,633.70,2099.43,4855.95
279,973,Guyane,292354.0,__pm2_cty_comp_3m,973,cty,ALL,DEPT,973,2025-09-01,...,973,2025-09-01,9.0,2107.49,1748.56,1883.12,611.01,633.70,2099.43,4855.95
280,974,La Réunion,896175.0,__pm2_cty_3m,MAI_974,cty,MAI,DEPT,974,2025-09-01,...,974,2025-09-01,11.0,1792.04,1337.76,1500.00,540.00,1325.68,1892.55,2200.00


In [32]:
df_6m_merged = df_merged[((df_merged['DIC_NAME_y'] == '__pm2_cty_6m') | (df_merged['DIC_NAME_y'] == '__pm2_cty_comp_6m'))]
df_prix_m2_dept= pd.merge(df_prix_m2, df_6m_merged[['Département', 'CD_TYP_BIEN', 'mean_6m']], on=['Département', 'CD_TYP_BIEN'], how='left')
df_prix_m2_dept

,Numéro,Département,Population,DIC_NAME,KEY_DICT,MAILLE_DICT,CD_TYP_BIEN,MAILLE_GEO,VAL_MAILLE_GEO,DT_MAJ_PM2,NB_BIENS,MEAN,STD,MEDIAN,D1,Q1,Q3,D9,mean_6m
0,01,Ain,688626.0,__pm2_cty_3m,APT_01,cty,APT,DEPT,01,2025-09-01,19.0,2108.27,1203.65,1563.95,1062.46,1263.93,3108.17,3678.47,1965.25
1,01,Ain,688626.0,__pm2_cty_3m,MAI_01,cty,MAI,DEPT,01,2025-09-01,28.0,2753.39,907.41,2853.71,1408.82,2164.81,3595.51,3708.63,2609.68
2,01,Ain,688626.0,__pm2_cty_6m,APT_01,cty,APT,DEPT,01,2025-09-01,8.0,1965.25,692.05,1734.97,1271.73,1555.56,2323.25,2890.60,1965.25
3,01,Ain,688626.0,__pm2_cty_6m,MAI_01,cty,MAI,DEPT,01,2025-09-01,71.0,2609.68,823.00,2669.90,1512.61,2142.26,3171.43,3601.77,2609.68
4,01,Ain,688626.0,__pm2_cty_12m,APT_01,cty,APT,DEPT,01,2025-09-01,50.0,2319.64,1393.15,1759.99,1118.83,1459.12,2714.89,4052.52,1965.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
841,974,La Réunion,896175.0,__pm2_cty_6m,MAI_974,cty,MAI,DEPT,974,2025-09-01,11.0,1792.04,1337.76,1500.00,540.00,1325.68,1892.55,2200.00,1792.04
842,974,La Réunion,896175.0,__pm2_cty_12m,MAI_974,cty,MAI,DEPT,974,2025-09-01,13.0,1631.03,1286.64,1478.26,512.04,985.92,1848.48,2147.32,1792.04
843,974,La Réunion,896175.0,__pm2_cty_comp_3m,974,cty,ALL,DEPT,974,2025-09-01,11.0,1792.04,1337.76,1500.00,540.00,1325.68,1892.55,2200.00,1792.04
844,974,La Réunion,896175.0,__pm2_cty_comp_6m,974,cty,ALL,DEPT,974,2025-09-01,11.0,1792.04,1337.76,1500.00,540.00,1325.68,1892.55,2200.00,1792.04


In [33]:
df_prix_m2_dept['mean_6m'] = df_prix_m2_dept['mean_6m'].where(df_prix_m2_dept['DIC_NAME'].str.endswith('6m'), np.nan)

df_6m = df[(df['DIC_NAME'] == '__pm2_cty_6m') & (df['CD_TYP_BIEN'] != 'ALL') | 
            (df['DIC_NAME'] == '__pm2_cty_comp_6m') & (df['CD_TYP_BIEN'] == 'ALL')]
df_12m = df[((df['DIC_NAME'] == '__pm2_cty_12m') & (df['CD_TYP_BIEN'] != 'ALL')) | 
            ((df['DIC_NAME'] == '__pm2_cty_comp_12m') & (df['CD_TYP_BIEN'] == 'ALL'))]


df_6m = df_6m.rename(columns={'MEAN': 'mean_6m', 'NB_BIENS': 'nb_bien_6m'})
df_12m = df_12m.rename(columns={'MEAN': 'mean_12m', 'NB_BIENS': 'nb_bien_12m'})
df_merged2 = pd.merge(df_6m, df_12m, on=['Département', 'CD_TYP_BIEN'], how='inner')


In [34]:
df_12m_merged = df_merged2[((df_merged2['DIC_NAME_y'] == '__pm2_cty_12m') | (df_merged2['DIC_NAME_y'] == '__pm2_cty_comp_12m'))]
df_prix_m2_dept = pd.merge(df_prix_m2_dept, df_12m_merged[['Département', 'CD_TYP_BIEN', 'mean_12m']], on=['Département', 'CD_TYP_BIEN'], how='left')
df_prix_m2_dept['mean_12m'] = df_prix_m2_dept['mean_12m'].where(df_prix_m2_dept['DIC_NAME'].str.endswith('12m'), np.nan)

In [35]:
table = df_prix_m2_dept[['Département', 'CD_TYP_BIEN', 'mean_6m', 'mean_12m', 'DIC_NAME', 'MEAN', 'NB_BIENS', 'VAL_MAILLE_GEO', 'STD']]
list_tables.append(table)
table_final = pd.concat(list_tables, ignore_index=True)


with open(r'N:\uflumth\indicateur_marche\prix_m2\dep_fr.geojson') as f:
    geojson = json.load(f)


types_bien = ['ALL', 'APT', 'MAI']

for type_bien in types_bien:

    requete_sql = f"""
    SELECT  DISTINCT *
    FROM table_final
    WHERE CD_TYP_BIEN = '{type_bien}' AND DIC_NAME LIKE '%cty_%_%m' AND mean_12m NOT NULL"""
    df = ps.sqldf(requete_sql, locals())

    for feature in geojson['features']:
        code = feature['properties']['code']
        row = df.loc[df['VAL_MAILLE_GEO'] == code]
        for col in df.columns:
            if col == 'mean_12m' and len(row[col].values) > 0:
    
                feature['properties'][col] = f"Prix au m² : {round(row[col].values[0])} €"
            elif len(row[col].values) > 0:
                feature['properties'][col] = row[col].values[0]
            else:
                feature['properties'][col] = None

    m = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

    
    choropleth = folium.Choropleth(
        geo_data=geojson,
        name='choropleth',
        data=df,
        columns=['VAL_MAILLE_GEO', 'mean_12m'],
        key_on='feature.properties.code',
        fill_color='YlGn',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='Prix au m²'
    ).add_to(m)

   
    choropleth.geojson.add_child(
        folium.features.GeoJsonTooltip(['nom', 'mean_12m'], labels=False)
    )

    m.save(rf'N:\uflumth\indicateur_marche\prix_m2\ma_carte_prix_m2_dept_{type_bien}.html')

### Tableau récapitulatif

In [36]:
departments_to_keep = ['75', '77', '78', '91', '92', '93', '94', '95', '59', '69', '06', '13'] # A changer si nécessaire
n_plus_grandes_villes = 15 # nombre de plus grandes villes à conserver dans le tableau

In [37]:
df_final = pd.DataFrame()
list_tables = [] 


requete_sql = f"""
SELECT  *
FROM df_prix_m2"""
df_prix_m2_dept = ps.sqldf(requete_sql, locals())

df_prix_m2_dept = df_prix_m2_dept[df_prix_m2_dept['Numéro'].isin(departments_to_keep)]
df = df_prix_m2_dept

df_3m = df[(df['DIC_NAME'] == '__pm2_cty_3m') & (df['CD_TYP_BIEN'] != 'ALL') | 
            (df['DIC_NAME'] == '__pm2_cty_comp_3m') & (df['CD_TYP_BIEN'] == 'ALL')]
df_6m = df[((df['DIC_NAME'] == '__pm2_cty_6m') & (df['CD_TYP_BIEN'] != 'ALL')) | 
            ((df['DIC_NAME'] == '__pm2_cty_comp_6m') & (df['CD_TYP_BIEN'] == 'ALL'))]

df_3m = df_3m.rename(columns={'MEAN': 'mean_3m', 'NB_BIENS': 'nb_bien_3m'})
df_6m = df_6m.rename(columns={'MEAN': 'mean_6m', 'NB_BIENS': 'nb_bien_6m'})
df_merged = pd.merge(df_3m, df_6m, on=['Département', 'CD_TYP_BIEN'], how='inner')



df_6m_merged = df_merged[((df_merged['DIC_NAME_y'] == '__pm2_cty_6m') | (df_merged['DIC_NAME_y'] == '__pm2_cty_comp_6m'))]
df_prix_m2_dept= pd.merge(df_prix_m2_dept, df_6m_merged[['Département', 'CD_TYP_BIEN', 'mean_6m']], on=['Département', 'CD_TYP_BIEN'], how='left')
df_prix_m2_dept['mean_6m'] = df_prix_m2_dept['mean_6m'].where(df_prix_m2_dept['DIC_NAME'].str.endswith('6m'), np.nan)

In [38]:

df_6m = df[(df['DIC_NAME'] == '__pm2_cty_6m') & (df['CD_TYP_BIEN'] != 'ALL') | 
            (df['DIC_NAME'] == '__pm2_cty_comp_6m') & (df['CD_TYP_BIEN'] == 'ALL')]
df_12m = df[((df['DIC_NAME'] == '__pm2_cty_12m') & (df['CD_TYP_BIEN'] != 'ALL')) | 
            ((df['DIC_NAME'] == '__pm2_cty_comp_12m') & (df['CD_TYP_BIEN'] == 'ALL'))]

df_6m = df_6m.rename(columns={'MEAN': 'mean_6m', 'NB_BIENS': 'nb_bien_6m'})
df_12m = df_12m.rename(columns={'MEAN': 'mean_12m', 'NB_BIENS': 'nb_bien_12m'})
df_merged = pd.merge(df_6m, df_12m, on=['Département', 'CD_TYP_BIEN'], how='inner')



df_12m_merged = df_merged[((df_merged['DIC_NAME_y'] == '__pm2_cty_12m') | (df_merged['DIC_NAME_y'] == '__pm2_cty_comp_12m'))]
df_prix_m2_dept = pd.merge(df_prix_m2_dept, df_12m_merged[['Département', 'CD_TYP_BIEN', 'mean_12m']], on=['Département', 'CD_TYP_BIEN'], how='left')
df_prix_m2_dept['mean_12m'] = df_prix_m2_dept['mean_12m'].where(df_prix_m2_dept['DIC_NAME'].str.endswith('12m'), np.nan)

In [39]:
# TODO : ajouter Tableau des prix par mètre carré selon Crédit Logement : df_final



table = df_prix_m2_dept[['Département', 'CD_TYP_BIEN', 'mean_6m', 'mean_12m', 'DIC_NAME', 'MEAN', 'NB_BIENS', 'VAL_MAILLE_GEO', 'STD']]
list_tables.append(table)




def create_pivot(df, dic_name, value_column, index='Département', columns='CD_TYP_BIEN'):
    unique_cols = df[columns].unique()
    pivot = pd.DataFrame(columns=pd.MultiIndex.from_product([[f'prix_m2_{dic_name}'], unique_cols]), index=df[index].unique())
    subset_df = df[df['DIC_NAME'].str.contains(dic_name)]
    if not subset_df[value_column].isna().all():
        pivot_temp = subset_df.pivot_table(index=index, columns=columns, values=value_column)
        pivot_temp.columns = pd.MultiIndex.from_product([[f'prix_m2_{dic_name}'], pivot_temp.columns])
        pivot.update(pivot_temp)
    for col in pivot.columns:
        pivot[col] = pivot[col].apply(lambda x: f"{x:,.0f}".replace(',', ' ') + ' €' if pd.notna(x) else '-')
        
    return pivot



pivot_3m = create_pivot(df_prix_m2_dept, '3m', 'MEAN')
pivot_6m = create_pivot(df_prix_m2_dept, '6m', 'mean_6m')
pivot_12m = create_pivot(df_prix_m2_dept, '12m', 'mean_12m')


df_new = pd.concat([pivot_3m, pivot_6m, pivot_12m], axis=1)

if not df_new.empty:
    df_new = df_new.round(0)

    df_final = pd.concat([df_final, df_new], axis=0)

df_final.columns = df_final.columns.set_levels(['12 mois' if x == 'prix_m2_12m' else x for x in df_final.columns.levels[0]], level=0)
df_final.columns = df_final.columns.set_levels(['3 mois' if x == 'prix_m2_3m' else x for x in df_final.columns.levels[0]], level=0)
df_final.columns = df_final.columns.set_levels(['6 mois' if x == 'prix_m2_6m' else x for x in df_final.columns.levels[0]], level=0)
cols_non_12m = [col for col in df_final.columns if col[0] != '12 mois']
cols_12m = [col for col in df_final.columns if col[0] == '12 mois']
ordered_cols = cols_non_12m + cols_12m
df_final = df_final[ordered_cols]

print(df_final)

table_final = pd.concat(list_tables, ignore_index=True)
requete_sql = f"""
SELECT  DISTINCT *
FROM table_final
WHERE CD_TYP_BIEN = 'ALL' AND DIC_NAME LIKE '%cty_comp_%m' AND mean_12m NOT NULL"""
df = ps.sqldf(requete_sql, locals())


                   3 mois                     6 mois                    \
                      APT      MAI      ALL      APT      MAI      ALL   
Alpes-Maritimes   2 617 €  3 748 €  4 520 €  4 075 €  3 748 €  4 152 €   
Bouches-du-Rhône  3 808 €  4 162 €  4 162 €  3 808 €  4 162 €  4 162 €   
Nord              1 961 €  2 594 €  2 594 €  2 100 €  2 016 €  2 026 €   
Rhône             3 227 €  2 991 €  3 050 €  3 266 €  3 156 €  3 187 €   
Seine-et-Marne    3 368 €  2 352 €  2 473 €  3 317 €  2 410 €  2 522 €   
Yvelines          4 687 €  2 994 €  3 104 €  4 666 €  2 941 €  3 157 €   
Essonne           2 764 €  2 514 €  2 552 €  2 759 €  2 543 €  2 576 €   
Val-de-Marne            -        -        -        -        -  2 804 €   
Val-d'Oise        3 652 €  3 017 €  3 116 €  3 528 €  2 971 €  3 073 €   

                  12 mois                    
                      APT      MAI      ALL  
Alpes-Maritimes   2 536 €  3 177 €  2 971 €  
Bouches-du-Rhône  3 705 €  4 726 €  4 537 €  
N

### Meilleurs Agents

Réactualisées tous les mois pour coller à la réalité du marché, nos estimations de prix sont exprimées en net vendeur (hors frais d'agence et notaires).

Les bornes de la fourchette sont calculées pour qu'elle inclue 90% des prix du marché, en excluant les 5% des prix les plus faibles comme 5% des prix les plus élevés de la zone "France". Puis la moyenne des prix est gardée comme valeur référence

En Ile-de-France :

Les prix sont calculés par MeilleursAgents sur la base de deux sources d'informations complémentaires :
les transactions historiques enregistrées par la base BIEN des Notaires de Paris / Ile de France
les dernières transactions remontées par les agences immobilières partenaires de MeilleursAgents.
Hors Ile-de-France :

Les prix sont calculés par MeilleursAgents sur la base des données de transaction communiquées par nos agences partenaires, d'annonces immobilières et de données éco-socio-démographiques. Afin d'obtenir des prix de marché comparables en qualité à ceux communiqués en Ile-de-France, l'équipe scientifique de MeilleursAgents.com développe des moyens d'analyse et de traitement de l'information sophistiqués.
MeilleursAgents.com travaille en permanence à l'amélioration des sources de prix et des méthodes de calcul afin de fournir à tout moment les estimations immobilières les plus fiables et les plus transparentes.

In [46]:
df_MA_dep = pd.read_excel(r"N:\uflumth\indicateur_marche\prix_m2\Meilleurs agents données finales -  Septembre 2025.xlsx", sheet_name="Départements - Comp")
df_MA_dep = df_MA_dep.drop(df_MA_dep.index[0])

df_MA_dep['Prix m2 moyen (maisons)'] = df_MA_dep['Prix m2 moyen (maisons)'].astype(float)
df_MA_dep["Numéro département"] = df_MA_dep["Départements"].apply(lambda x : x.split('-')[0])
df_MA_dep = df_MA_dep.drop(columns = ['Loyer m2 moyen (maisons)','Loyer m2 moyen (appartements)'], errors='inplace')
df_MA_dep["Numéro département"] = df_MA_dep["Numéro département"].str.strip()
df_MA_dep["Date"] = pd.to_datetime(df_MA_dep["Date"])

df_MA_dep['Mois'] = df_MA_dep['Date'].dt.month
df_MA_dep['Année'] = df_MA_dep['Date'].dt.year


df_select = df_MA_dep[df_MA_dep['Numéro département'].isin(departments_to_keep)].copy()
df_select


,Date,Départements,Prix m2 moyen (appartements),Prix m2 moyen (maisons),Numéro département,Mois,Année
6,2025-09-01,06 - Alpes-Maritimes,5222.0,5439.0,06,9,2025
13,2025-09-01,13 - Bouches-du-Rhône,3642.0,4148.0,13,9,2025
60,2025-09-01,59 - Nord,2579.0,2065.0,59,9,2025
70,2025-09-01,69 - Rhône,3971.0,3788.0,69,9,2025
76,2025-09-01,75 - Paris,9751.0,10186.0,75,9,2025
...,...,...,...,...,...,...,...
1244,2024-09-01,91 - Essonne,3040.0,2943.0,91,9,2024
1245,2024-09-01,92 - Hauts-de-Seine,6545.0,7016.0,92,9,2024
1246,2024-09-01,93 - Seine-Saint-Denis,4127.0,3492.0,93,9,2024
1247,2024-09-01,94 - Val-de-Marne,5139.0,4523.0,94,9,2024


In [47]:
date_max = df_select['Date'].max()
date_min_3m = date_max - pd.DateOffset(months = 3)
date_min_6m = date_max - pd.DateOffset(months = 6)
date_min_12m = date_max - pd.DateOffset(months = 12)

df_3m = df_select[df_select['Date'] > date_min_3m].groupby("Départements")["Prix m2 moyen (appartements)"].mean()
df_6m = df_select[df_select['Date'] > date_min_6m].groupby("Départements")["Prix m2 moyen (appartements)"].mean()
df_12m = df_select[df_select['Date'] > date_min_12m].groupby("Départements")["Prix m2 moyen (appartements)"].mean()

df_moyennes = pd.DataFrame({'Moyenne 3 derniers mois': df_3m,
                            'Moyenne 6 derniers mois': df_6m,
                            'Moyenne 12 derniers mois': df_12m,
                            })
print(df_moyennes)



                        Moyenne 3 derniers mois  Moyenne 6 derniers mois  \
Départements                                                               
06 - Alpes-Maritimes                5178.133333              5156.233333   
13 - Bouches-du-Rhône               3639.700000              3618.683333   
59 - Nord                           2564.933333              2524.966667   
69 - Rhône                          3948.600000              3873.133333   
75 - Paris                          9668.633333              9568.483333   
77 - Seine-et-Marne                 3265.300000              3311.150000   
78 - Yvelines                       4575.900000              4558.616667   
91 - Essonne                        3054.600000              3069.966667   
92 - Hauts-de-Seine                 6571.566667              6547.116667   
93 - Seine-Saint-Denis              4021.066667              4069.866667   
94 - Val-de-Marne                   5070.700000              5109.350000   
95 - Val-d'O

### Comparaison